### 파킨슨병 데이터
- 환자들의 뇌를 촬영한 사진의 상태를 기록한 자료에 각 환자의 상태 status(1: 파킨슨병 진단, 0: 파킨슨병 아님)로 추가한 테이블
- (data/parkinsons.csv)
1. 파킨슨 병을 예측하는 모델로 로지스틱 회귀모형을 적용하여 생성
2. 파킨슨병을 예측하는데 영향을 미치는 변수를 중요한 순서대로 3개 선정
3. 파킨슨 병을 진단하는 기준를 함수로 생성하여(매개변수명 = threshold, 함수명 = cutoff)을 0.5로 했을 때와 0.8로 했을 때 F1-스코어를 비교
    - 분석 조건
        - 필요 없는 컬럼 name을 삭제
        - 데이터의 정규화는 min-max 스케일러 사용
        - 로지스틱 회귀를 위한 상수항 추가
        - status는 카테고리 타입으로 변환
        - 트레이닝셋과 테스트셋 비율은 9:1
        - 모델은 로지스틱 회귀분석 사용
        - 모델의 최적화 방법론은 "bfgs" 사용

In [45]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.preprocessing import Binarizer

In [33]:
data = pd.read_csv('./parkinsons.csv')

In [34]:
data = data.drop('name',axis=1)

In [35]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.drop('status', axis=1))
scaled_data = pd.DataFrame(scaled_data, columns=data.columns[:-1])

In [36]:
scaled_data['intercept'] = 1
scaled_data['status'] = data['status'].astype('category')

In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    scaled_data.drop('status', axis=1),
    scaled_data['status'],
    test_size=0.1,
    random_state=42
)

In [38]:
Scaler=MinMaxScaler()
Scaler.fit(X_train)
x_train = Scaler.transform(X_train)

x_train.max()

1.0

In [39]:
logistic=LogisticRegression(multi_class='multinomial', solver='lbfgs',fit_intercept=True)
logistic.fit(x_train,y_train)

LogisticRegression(multi_class='multinomial')

In [40]:
Scaler.fit(X_test)
x_test=Scaler.transform(X_test)

x_test.max()

1.0000000000000002

In [41]:
sorted_features = np.argsort(logistic.coef_)
top_3_features = sorted_features[0, -3:]
top_3_features

array([19, 20, 17])

In [42]:
data.drop('status',axis=1).columns[top_3_features]

Index(['spread2', 'D2', 'DFA'], dtype='object')

In [46]:
pred_proba=logistic.predict_proba(x_test)
pred_proba1=pred_proba[:,1].reshape(-1,1)

def cutoff(threshold):
    binarizer=Binarizer(threshold=threshold).fit(pred_proba1)
    custom_predict=binarizer.transform(pred_proba1)

    F1_score=f1_score(y_test,custom_predict)
    return F1_score

In [47]:
print(cutoff(0.5))

0.8888888888888888


In [48]:
print(cutoff(0.8))

0.8
